In [2]:
from datetime import datetime

datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2023-11-13 11:49:28'

- UpdateCryDuration: 울음이 감지되어 예측이 종료되었음에도 또 다시 울음이 감지되면(1분 이내) 이전 예측 시간을 갱신한다.


### 울음소리 분석

0. 오늘 울음 횟수 분석(일간)

   - 오늘 발생한 울음을 분석한다.

   - 각각 설명을 보여준다.

   - 새로운 페이지로 넘어가서 보여준다.(2주일 이전의 내용을 보고 싶다면 프리미엄을 구매해야 함.)

1. 아기가 주로 우는 시간대 분석(월간)

   - 울음 분도를 통해 하루 중 가장 울음이 많이 발생하는 시간대를 분석한다.

   - x축에 시간, y축은 울음 빈도로 하는 히스토그램을 그린다.

2. 일별 울음 빈도 분석(월간)

   - 일별 울음 빈도를 분석한다.

   - x축에 날짜, y축은 울음 빈도로 하는 선 그래프를 그린다.

3. 아기 울음 원인 분석(월간)

   - 아기의 울음 원인 빈도를 분석한다.

   - 7가지의 울음 원인 횟수와 비율을 보여주는 파이 차트를 그린다.

4. 아기 울음 원인에 따른 울음 지속 시간 분석

   - 아기의 울음 원인에 따른 울음 지속 시간을 분석한다.

   - 울음 예측 확률 표시하는 바와 같이 각각의 울음 원인에 따른 울음 지속 시간을 표시하는 가로 막대 그래프.


In [152]:
import random

# Rewriting the generate_predict_map function as per the new requirements


def generate_predict_map():
    types = ['sad', 'hug', 'diaper', 'hungry',
             'sleepy', 'awake', 'uncomfortable']

    # Randomly select 4 types to have a probability of 0
    zero_probability_types = random.sample(types, 4)

    # Remaining types will have probabilities of 0.8, 0.1, and 0.1
    remaining_types = [t for t in types if t not in zero_probability_types]
    probabilities = [0.85, 0.1, 0.05]
    uniform = random.uniform(-0.07, 0.07)
    half_uniform = uniform / 2
    probabilities[0] -= uniform
    probabilities[1] += half_uniform
    probabilities[2] += half_uniform
    print(probabilities)
    # Shuffle to assign these probabilities randomly
    random.shuffle(probabilities)

    # Combine into a single dictionary
    predict_map = {t: 0.0 for t in zero_probability_types}
    for t, p in zip(remaining_types, probabilities):
        predict_map[t] = round(p, 3)

    return predict_map


# Testing the function
generate_predict_map_sample = generate_predict_map()
generate_predict_map_sample

[0.838580450501035, 0.10570977474948248, 0.05570977474948247]
1.001


{'diaper': 0.0,
 'sad': 0.0,
 'hug': 0.0,
 'hungry': 0.0,
 'sleepy': 0.839,
 'awake': 0.106,
 'uncomfortable': 0.056}

In [154]:
import json
import random
from datetime import datetime, timedelta


def save_cry_state_sample_data_to_file(start_date, end_date, baby_id, file_path):
    # Constants
    types = ['sad', 'hug', 'diaper', 'hungry',
             'sleepy', 'awake', 'uncomfortable']
    intensities = ['low', 'medium', 'high']
    audio_id_counter = 1

    def generate_predict_map():
        # Randomly select 4 types to have a probability of 0
        zero_probability_types = random.sample(types, 4)

        # Remaining types will have probabilities of 0.8, 0.1, and 0.1
        remaining_types = [t for t in types if t not in zero_probability_types]
        probabilities = [0.85, 0.1, 0.05]
        uniform = random.uniform(-0.07, 0.07)
        half_uniform = uniform / 2
        probabilities[0] -= uniform
        probabilities[1] += half_uniform
        probabilities[2] += half_uniform
        # Shuffle to assign these probabilities randomly
        random.shuffle(probabilities)

        # Combine into a single dictionary
        predict_map = {t: 0.0 for t in zero_probability_types}
        for t, p in zip(remaining_types, probabilities):
            predict_map[t] = round(p, 3)

        return predict_map

    # Writing data to the file
    with open(file_path, 'w') as file:
        current_date = start_date
        while current_date <= end_date:
            # Random number of records for the day
            num_records = random.randint(1, 4)
            for _ in range(num_records):
                # Generate random time for the current date
                random_time = current_date + timedelta(hours=random.randint(
                    0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))
                # Randomly select intensity and duration
                intensity = random.choice(intensities)
                duration = round(random.uniform(2.0, 17.0), 2)
                # Generate predictMap and select type with the largest probability
                predict_map = generate_predict_map()
                selected_type = max(predict_map, key=predict_map.get)
                # Create the SQL INSERT statement
                insert_statement = f"INSERT INTO cry_state (babyId, time, type, audioId, predictMap, intensity, duration) VALUES ('{baby_id}', '{random_time.strftime('%Y-%m-%d %H:%M:%S')}', '{selected_type}', 'audioId{audio_id_counter}', '{json.dumps(predict_map)}', '{intensity}', {duration});\n"
                file.write(insert_statement)
                audio_id_counter += 1
            current_date += timedelta(days=1)

    return file_path


# Example usage
file_path = 'cry_state_sample_data.txt'
start_date = datetime(2023, 9, 12)
end_date = datetime(2023, 11, 27)
baby_id = "355cf5e3-71a3-4807-885a-441620a5efc8"
save_cry_state_sample_data_to_file(start_date, end_date, baby_id, file_path)

'cry_state_sample_data.txt'